## STEP 0 - install dependencies

In [2]:
# we have to enter OpenAI key here and then we can run all cells freely...
import os
from getpass import getpass
from openai import OpenAI

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key


In [3]:
# source is https://www.youtube.com/watch?v=GH3lrOsU3AU
# and notebook is https://github.com/DataTalksClub/llm-zoomcamp/blob/main/0a-agents/notebook.ipynb
# Follow along this tutorial: https://github.com/alexeygrigorev/rag-agents-workshop

# STEP 0 - installing packages we need here and in the VS code terminal:

# Run in VS code terminal:
# pip install --upgrade pip # no reminders after!
# pip install tqdm notebook==7.1.2 openai elasticsearch==8.13.0 pandas scikit-learn ipywidgets
# jupyter notebook # to run jupyter engine locally

In [4]:
%pip install minsearch -q

Note: you may need to restart the kernel to use updated packages.


## STEP 1 - download evaluation data from Github and build in-memory search index with minsearch

In [5]:
# download our FAQ dataset
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

documents[-1] # to see a format of downloaded FAQ dataset

{'text': 'Problem description\nInfrastructure created in AWS with CD-Deploy Action needs to be destroyed\nSolution description\nFrom local:\nterraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" --reconfigure\nterraform destroy --var-file vars/prod.tfvars\nAdded by Erick Calderin',
 'section': 'Module 6: Best practices',
 'question': 'How to destroy infrastructure created via GitHub Actions',
 'course': 'mlops-zoomcamp'}

In [6]:
# build index - takes a few seconds with minsearch

from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [7]:
# create a search function with weights: boost = {'question': 3.0, 'section': 0.5}

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

# number of results 5 and course is hard coded here  - filter_dict={'course': 'data-engineering-zoomcamp'}

In [8]:
question = 'Can I still join the course?'

In [9]:
# test if our search function works 

search(question)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  '_id': 2},
 {'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",
  'section': 'General course-related questions',
  'question': 'Certificate - Can I follow the course in a self-paced mode and get a certificate?',
  'course': 'data-engineering-zoomcamp',
  '_id': 11},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the cou

## STEP 2 - build prompt

In [10]:
# build a prompt

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt



In [11]:
# test how our prompt build works by calling search(question)

prompt = build_prompt(question, search(question))
print(prompt) # better formatted for readability

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT>
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.

section: General

## STEP 3 - Connect to OpenAI

In [12]:
# connecting to LLM, we entered our API KEY already at the first cell
client = OpenAI()

In [13]:
search_results = search(question)
# just repeating step 1 - keyword search 

In [14]:
prompt = build_prompt(question, search_results)
# repeating step 2 - building prompt on top of search results 

In [15]:
# function to send our prompt to llm 

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content



In [16]:
# test if our llm can answer something meaningful based on prompt provided

answer = llm(prompt)
print(answer)

Yes, you can still join the course even after the start date. Even if you don't register, you're eligible to submit the homework. However, be aware that there will be deadlines for turning in the final projects, so it's advisable not to leave everything to the last minute.


In [17]:
print(prompt) # this is what we have sent to llm

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT>
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.

section: General

## STEP 4 - assemble our RAG pipeline

In [18]:
# this is our RAG pipeline

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer



In [19]:
# test by irrelevant question - something which is NOT in FAQ

rag("How do I patch KDE under FreeBSD?")

"I'm sorry, but it appears that there is no relevant information provided in the context regarding how to patch KDE under FreeBSD. Please check the official documentation or community forums for guidance on this topic."

In [20]:
rag("What Shakespeare said about peeling a carrot?") 

'The context provided does not contain any information regarding what Shakespeare said about peeling a carrot. Therefore, I am unable to answer that question.'

In [21]:
# relevant question - it actually was in FAQ
answer = rag("How to run Kafka in Docker")
print(answer)

To run Kafka in Docker, first ensure that your Kafka broker Docker container is working. You can confirm this by running the command `docker ps`. If the container is not running, navigate to the folder containing your Docker Compose YAML file and execute `docker compose up -d` to start all the instances. This will set up the necessary services for Kafka to run.


In [22]:
# LLM by itself know the answer, but our RAG is prohibiting it
# above rag("How do I patch KDE under FreeBSD?") returned nothing, but - 
print(llm("How do I patch KDE under FreeBSD?"))

Patching KDE under FreeBSD involves several steps, including downloading the relevant patches, applying them to the source code, and then rebuilding the software. Below is a general guide on how to do this:

### Step 1: Install Required Tools
You should ensure you have the necessary tools for building software on FreeBSD. You can do this by installing the FreeBSD ports collection and other essential tools.

You can install the `ports` tree if you haven't already:
```sh
pkg install portsnap
portsnap fetch update
```

### Step 2: Download KDE Ports
If you want to patch a specific version of KDE, navigate to the relevant ports directory. For example, if you're looking to patch KDE Plasma, you'd go to the `x11/kde5` directory:
```sh
cd /usr/ports/x11/kde5
```

### Step 3: Applying the Patch
1. **Download the Patch:** Acquire the patch you want to apply. This might be from the KDE website, a Git repository, or a mailing list. The patch file will typically be a `.diff` or `.patch` file.

2. 

In [23]:
# Actually this was the main idea for STEP 5 below - if llm can answer question by itself, nice!
# if not - it should be able to search our FAQ database and build the context for answer...

## STEP 5 - "Agentic" RAG

In [24]:
# essentially we only modify our prompt - so llm can decide either to give an answer immediately
# or use a SEARCH tool to get more CONTEXT from our FAQ

prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [25]:
question = 'Can I still join the course?'
context = 'EMPTY'

In [26]:
prompt = prompt_template.format(question=question, context=context)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}

If you can answer the QUESTION using CONTEXT, use this template:

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}

If the context doesn't contain the answer, use your own knowledge to answer the question

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}


In [27]:
answer_json = llm(prompt)
answer_json 
# model decided to use "SEARCH" function because CONTEXT is empty
# and provided a reason for it  - "reasoning": "I am unsure about the specific enrollment dates...

'{\n"action": "SEARCH",\n"reasoning": "The context is empty, and I need to verify if there are any specific deadlines or requirements for joining the course. Checking the FAQ database will provide accurate information regarding course enrollment."\n}'

In [28]:
import json
# we can parse the llm answer and use tool if appropriate:

answer = json.loads(answer_json)
answer['action']

'SEARCH'

In [29]:
# lets ask LLM something it knows already well:
question = 'Can I run Docker on Windows 10?'
context = 'EMPTY'
prompt = prompt_template.format(question=question, context=context)

In [30]:
answer_json = llm(prompt)
answer_json 
# it provides answer immediately - zero shot - action": "ANSWER":
# and says it knows it already - "source": "OWN_KNOWLEDGE"

'{\n"action": "ANSWER",\n"answer": "Yes, you can run Docker on Windows 10. Docker provides a version called Docker Desktop that is specifically designed for Windows 10. You\'ll need to ensure that your system meets the requirements, such as having a supported edition of Windows (Pro, Enterprise, or Education) and that virtualization is enabled in your BIOS. Once you install Docker Desktop, you\'ll be able to create and manage containers directly on your Windows environment.",\n"source": "OWN_KNOWLEDGE"\n}'

In [31]:
answer = json.loads(answer_json)
answer['action']

'ANSWER'

In [32]:
# if model says SEARCH then we need to use a search tool and build context + update prompt

def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return context.strip()



In [33]:
# asking something it should search for:
question = 'Can I still join the course?'

search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)
print(prompt) # for better formatting

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT> 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the cour

In [34]:
# lets ask llm after we performed the search as advised and updated llm context

answer_json = llm(prompt)
print(answer_json)

{
"action": "ANSWER",
"answer": "Yes, you can still join the course even after the start date. You'll be able to submit homework, but please keep in mind that there will be deadlines for final projects, so it's best not to leave your work until the last minute.",
"source": "CONTEXT"
}


In [35]:
# we need to put everything together and parse our response json automatically
# algo - if "action": "SEARCH" then llm uses our FAQ search tool
# if "action": "ANSWER" - then it answers immediately ...

def agentic_rag_v1(question):
    context = "EMPTY"
    prompt = prompt_template.format(question=question, context=context)
    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(answer)

    if answer['action'] == 'SEARCH':
        print('need to perform search...')
        search_results = search(question)
        context = build_context(search_results)
        
        prompt = prompt_template.format(question=question, context=context)
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        print(answer)

    return answer



In [36]:
# test our agentic RAG function

agentic_rag_v1('how do I join the course?')
# 'action': 'SEARCH', 'reasoning': 'The context is empty...

# 'action': 'ANSWER'
# 'source': 'OWN_KNOWLEDGE'

{'action': 'ANSWER', 'answer': "To join the course, please visit the course website and look for the enrollment section. You'll typically need to create an account or log in if you already have one. Then, follow the steps to register for the course you are interested in. If you have any issues, you can contact the course administration for further assistance.", 'source': 'OWN_KNOWLEDGE'}


{'action': 'ANSWER',
 'answer': "To join the course, please visit the course website and look for the enrollment section. You'll typically need to create an account or log in if you already have one. Then, follow the steps to register for the course you are interested in. If you have any issues, you can contact the course administration for further assistance.",
 'source': 'OWN_KNOWLEDGE'}

In [37]:
agentic_rag_v1('how patch KDE under FreeBSD?')
# 'reasoning': 'The student is asking for a specific method to patch KDE under FreeBSD, and the context is currently empty...

# 'action': 'ANSWER'
# 'source': 'OWN_KNOWLEDGE'

{'action': 'ANSWER', 'answer': 'To patch KDE under FreeBSD, you typically follow these steps: 1. Ensure you have the necessary tools installed, such as `patch`, `make`, and `git`. 2. Download the KDE source code you want to patch. You can find it in the FreeBSD ports collection or from the KDE repositories. 3. Obtain the patch file you want to apply. This could be a patch provided by a bug report, or one you have created yourself. 4. Navigate to the directory where the KDE source code is located. 5. Apply the patch using the command: `patch < /path/to/your/patchfile.patch`. 6. After applying the patch, compile and install the updated KDE software using the usual `make install` procedure. Make sure to test the updated version to ensure your patch works as intended.', 'source': 'OWN_KNOWLEDGE'}


{'action': 'ANSWER',
 'answer': 'To patch KDE under FreeBSD, you typically follow these steps: 1. Ensure you have the necessary tools installed, such as `patch`, `make`, and `git`. 2. Download the KDE source code you want to patch. You can find it in the FreeBSD ports collection or from the KDE repositories. 3. Obtain the patch file you want to apply. This could be a patch provided by a bug report, or one you have created yourself. 4. Navigate to the directory where the KDE source code is located. 5. Apply the patch using the command: `patch < /path/to/your/patchfile.patch`. 6. After applying the patch, compile and install the updated KDE software using the usual `make install` procedure. Make sure to test the updated version to ensure your patch works as intended.',
 'source': 'OWN_KNOWLEDGE'}

## STEP 6 - Agentic Search

Part 2: Agentic search

So far we had two actions only: search and answer.

But we can let our "agent" formulate one or more search queries - and do it for a few iterations until we found an answer

Let's build a prompt:

List available actions:

Search in FAQ

-- Answer using own knowledge

-- Answer using information extracted from FAQ

-- Provide access to the previous actions

Have clear stop criteria (no more than X iterations)

We also specify the output format, so it's easier to parse it

In [38]:
# deduplication function

def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result



In [39]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [40]:
# repetitive task - search, update context and search again if needed + delete duplicate results

question = 'how do I do well on module 1'
max_iterations = 3
iteration_number = 0
search_queries = []
search_results  = []
previous_actions = []



In [41]:
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current iteration number

In [42]:
answer_json = llm(prompt)
answer_json

'{\n"action": "SEARCH",\n"reasoning": "The question asks for guidance on how to perform well in a specific module. I need to find tips or guidelines related to success in Module 1 from the FAQ database.",\n"keywords": ["how to succeed in Module 1", "tips for Module 1", "Module 1 study strategy"]\n}'

In [43]:
answer = json.loads(answer_json)
answer

{'action': 'SEARCH',
 'reasoning': 'The question asks for guidance on how to perform well in a specific module. I need to find tips or guidelines related to success in Module 1 from the FAQ database.',
 'keywords': ['how to succeed in Module 1',
  'tips for Module 1',
  'Module 1 study strategy']}

In [44]:
previous_actions.append(answer)
keywords = answer['keywords']
keywords

['how to succeed in Module 1', 'tips for Module 1', 'Module 1 study strategy']

In [45]:
# now we have to search each keyword and add results to a context

for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [46]:
# lets see what we found

search_results = dedup(search_results)
search_results

[{'text': 'You need to look for the Py4J file and note the version of the filename. Once you know the version, you can update the export command accordingly, this is how you check yours:\n` ls ${SPARK_HOME}/python/lib/ ` and then you add it in the export command, mine was:\nexport PYTHONPATH=”${SPARK_HOME}/python/lib/Py4J-0.10.9.5-src.zip:${PYTHONPATH}”\nMake sure that the version under `${SPARK_HOME}/python/lib/` matches the filename of py4j or you will encounter `ModuleNotFoundError: No module named \'py4j\'` while executing `import pyspark`.\nFor instance, if the file under `${SPARK_HOME}/python/lib/` was `py4j-0.10.9.3-src.zip`.\nThen the export PYTHONPATH statement above should be changed to `export PYTHONPATH="${SPARK_HOME}/python/lib/py4j-0.10.9.3-src.zip:$PYTHONPATH"` appropriately.\nAdditionally, you can check for the version of ‘py4j’ of the spark you’re using from here and update as mentioned above.\n~ Abhijit Chakraborty: Sometimes, even with adding the correct version of p

In [47]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [48]:
answer_json = llm(prompt)
answer = json.loads(answer_json)
print(answer)

{'action': 'SEARCH', 'reasoning': 'The previous search did not yield relevant information for succeeding in Module 1, Docker and Terraform. I need to explore different queries that specifically target strategies and tips for success in this module.', 'keywords': ['success tips Module 1 Docker Terraform', 'study strategies Module 1', 'best practices Module 1 Docker Terraform']}


In [49]:
# now lets create a function which will perform a search in a loop

question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []

iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()


ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current 

In [50]:
print(answer['answer']) # for better look

To be successful in Module 1 (Docker and Terraform), it’s essential to grasp the foundational concepts of both tools. Here are some tips:

1. **Understanding Docker**: Familiarize yourself with Docker's architecture, including concepts like images, containers, and Dockerfiles. Practice building and running your own Docker containers.

2. **Terraform Basics**: Learn how Terraform works, including how to write and apply configuration files. Explore its resource management capabilities for infrastructure as code.

3. **Hands-on Practice**: Implement small projects using Docker and Terraform, such as containerizing a simple application and defining cloud resources to deploy it.

4. **Error Resolution**: Be prepared to troubleshoot common issues. For example, dealing with module errors (like psycopg2) often involves correcting dependencies through pip or conda.

5. **Engage with the Community**: Utilize forums, study groups, or Q&A platforms to ask questions and share knowledge with peers.


In [51]:
iteration

3

In [52]:
# put together agentic search as a function

def agentic_search(question):
    search_queries = []
    search_results = []
    previous_actions = []

    iteration = 0
    
    while True:
        print(f'ITERATION #{iteration}...')
    
        context = build_context(search_results)
        prompt = prompt_template.format(
            question=question,
            context=context,
            search_queries="\n".join(search_queries),
            previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
            max_iterations=3,
            iteration_number=iteration
        )
    
        print(prompt)
    
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        print(json.dumps(answer, indent=2))

        previous_actions.append(answer)
    
        action = answer['action']
        if action != 'SEARCH':
            break
    
        keywords = answer['keywords']
        search_queries = list(set(search_queries) | set(keywords))

        for k in keywords:
            res = search(k)
            search_results.extend(res)
    
        search_results = dedup(search_results)
        
        iteration = iteration + 1
        if iteration >= 4:
            break
    
        print()

    return answer



In [53]:
# and test our agentic search function

answer = agentic_search('how do I prepare for the course?')


ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current 

In [54]:
# for better formatting

print(answer)

{'action': 'ANSWER_CONTEXT', 'answer': "To prepare for the course, you can start by installing and setting up all necessary dependencies and requirements. These include a Google Cloud account, Google Cloud SDK, Python 3 (installed with Anaconda), Terraform, and Git. Additionally, reviewing the prerequisites and syllabus will help ensure you are comfortable with the subjects covered in the course. It's also a good idea to familiarize yourself with the course materials and assignments to get a head start.", 'source': 'CONTEXT'}


In [55]:
# no need of parsing json?
# answer = json.loads(answer)

print(answer['answer'])


To prepare for the course, you can start by installing and setting up all necessary dependencies and requirements. These include a Google Cloud account, Google Cloud SDK, Python 3 (installed with Anaconda), Terraform, and Git. Additionally, reviewing the prerequisites and syllabus will help ensure you are comfortable with the subjects covered in the course. It's also a good idea to familiarize yourself with the course materials and assignments to get a head start.


## STEP 7 - Function calling ("tool use")

Part 3: Function calling - code from
https://github.com/alexeygrigorev/rag-agents-workshop/tree/main

Function calling in OpenAI

We put all this logic inside our prompt.

But OpenAI and other providers provide a convenient API for adding extra functionality like search.

https://platform.openai.com/docs/guides/function-calling

It's called "function calling" - you define functions that the model can call, and if it decides to make a call, it returns structured output for that.

For example, let's take our search function:

In [56]:
# search tool

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results



In [57]:
answer = search("How to run Kafka in Docker")
print(answer)

[{'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.", 'section': 'Module 6: streaming with kafka', 'question': 'Module “kafka” not found when trying to run producer.py', 'course': 'data-engineering-zoomcamp', '_id': 372}, {'text': "Below I have listed some steps I took to rectify this and potentially other minor errors, in Windows:\nUse the git bash terminal in windows.\nActivate python venv from git

In [58]:
# how to describe our search function for OpenAI / Pydantic:
# (this is a json wrapper to use our minsearch function = search tool from above ))

search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

# "name": "search" - based on this Python will call search(query) from above
# and pass a property "query": { "type": "string" ... when / into this function call object is created?

